<a href="https://colab.research.google.com/github/rubyvanrooyen/MaxE/blob/master/RSS_SNR_calculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [181]:
from astropy import units as u
import numpy as np

# Defaults values

In [182]:
import ipywidgets
from ipywidgets import Layout

# observational setup
def obs_input(name_, description_, label_, value_, unit_):
    description = ipywidgets.Text(value=description_, layout=Layout(width='50%'), disabled=True)
    textlayout = ipywidgets.Layout(width='auto', height='auto') #set width and height
    label = ipywidgets.Text(value=label_, layout = textlayout, disabled=True)
    value = ipywidgets.Text(value=value_, placeholder=name_, layout = textlayout, disabled=False)
    unit = ipywidgets.Text(value=unit_, layout = textlayout, disabled=True)
    return ipywidgets.VBox([description,
                            ipywidgets.HBox([label, value, unit])])

phi_s = obs_input('phi_s', 'nominal slit width', 'phi_s', '1.5', 'arcseconds')
fwhm_0 = obs_input('fwhm_0', 'full width at half maximum @ zenith in Sutherland', 'fwhm_0', '1.2', 'arcseconds')
z_deg = obs_input('z_deg', 'azimuth offset angle', 'z_deg', '37', 'degree')
err = obs_input('err', 'assume an err imaging error budget', 'err', '0.6', 'arcseconds')
observation_input = ipywidgets.VBox([phi_s, fwhm_0, z_deg, err])

# instrumental parameters
def text_input(name_, label_, value_, unit_):
    textlayout = ipywidgets.Layout(width='auto', height='auto') #set width and height
    text_label = ipywidgets.Text(value=label_, layout = textlayout, disabled=True)
    text_value = ipywidgets.Text(value=value_, placeholder=name_, layout = textlayout, disabled=False)
    unit_label = ipywidgets.Text(value=unit_, layout = textlayout, disabled=True)
    return ipywidgets.HBox([text_label, text_value, unit_label])

f_tel = text_input('f_tel', 'telescope focal length', '46200', 'mm')
f_col = text_input('f_col', 'collimator focal length', '630', 'mm')
f_cam = text_input('f_cam', 'camera focal length', '330', 'mm')
spectograph_input = ipywidgets.VBox([f_tel, f_col, f_cam])

alpha = text_input('alpha', 'incident grating angle', '12.5', 'deg')
vg = text_input('vg', 'groove frequency', '900', 'grooves/mm')
m = text_input('m', 'spectral order', '1', '')
grating_input = ipywidgets.VBox([alpha, vg, m])

px = text_input('px', 'horizontal CCD pixel size', '15', 'um')
py = text_input('py', 'vertical CCD pixel size', '15', 'um')
bx = text_input('bx', 'horizontal CCD pixels binning', '2', 'pixels')
by = text_input('by', 'vertical CCD pixels binning', '2', 'pixels')
detector_input = ipywidgets.VBox([px, py, bx, by])

# SALT FWHM
And the PSF in the cross-dispersion direction $ =  \frac{FWHM \cdot M}{S_f} = \frac{FWHM}{S_d}$

SALT is a fixed azimuth mount telescope and the seeing at zenith has to be degraded by the airmass contribution at $37^\circ$ away from zenith    
$FWHM_{z=37} = FWHM_{z=0}\sec^{0.6}(37^\circ)$

Spot size at focal plane assuming a telescope imaging budget of $0.6''$    
$FWHM_{rms} = \sqrt{FWHM^2 + 0.6^2}$

# User Input Parameters

In [183]:
from google.colab import widgets
t = widgets.TabBar(["Observational conditions", "Grating", "Spectograph", "Detector"])
with t.output_to(3): display(detector_input)
with t.output_to(2): display(spectograph_input)
with t.output_to(1): display(grating_input)
with t.output_to(0): display(observation_input)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Setup variables and units for calculation
Observational conditions
* slit width
* fwhm
* err

Instrument parameters
* Spectograph parameters in units of $mm$
* Grating parameters in radians and $mm$
* Detector pixel and binning sizes in $\mu m$

In [184]:
f_tel =  float(f_tel.children[1].value) * u.mm 
f_col =  float(f_col.children[1].value) * u.mm
f_cam = float(f_cam.children[1].value) * u.mm

alpha = np.radians(float(alpha.children[1].value)) * u.radian
vg = float(vg.children[1].value) / u.mm
# groove separation (mm)
d = 1./vg
m = float(m.children[1].value)
print('Groove spacing of {:.3} using spectral order {}'.format(d, m))

# size of a CCD pixel
px = float(px.children[1].value) * u.micrometer
py = float(py.children[1].value) * u.micrometer
# binning of CCD pixels
bx = float(bx.children[1].value)
by = float(by.children[1].value)

# nominal slit width (arcseconds)
phi_s = float(phi_s.children[1].children[1].value) * u.arcsec
print('Slid width {:.3}'.format(phi_s))
# full width at half maximum @ zenith in Sutherland
fwhm_0 = float(fwhm_0.children[1].children[1].value) * u.arcsec
# scale FWHM from zenith the fixed azimuth 37 deg away from zenith
z_rad = np.radians(float(z_deg.children[1].children[1].value))
# airmass correction
sec = lambda z: np.arccos(z)
airmass = sec(z_rad)**0.6
fwhm = fwhm_0 * airmass
# assume a 0.6'' imaging error budget
err = float(err.children[1].children[1].value) * u.arcsec
# spot size at focal plane
fwhm = np.sqrt((fwhm*fwhm).value + (err*err).value) * u.arcsec
print('Spot size at focal plane {:.3}'.format(fwhm))

Groove spacing of 0.00111 mm using spectral order 1.0
Slid width 1.5 arcsec
Spot size at focal plane 1.26 arcsec


# Simulation Input signals

Assume a continuum object signal of R=21.4 mag, plus sky contribution of R=20.9 mag    
Calculating sky input flux using Vega standards    
$F_* = F_{vega} \times 10^{-0.4m_*}$

In [ ]:
def mag2flux(magnitude):
    # convert magnitude to photon flux
    std_flux = 702.0  # erg/s/cm^2/A

    # Star flux relative to Vega
    flux = std_flux * (10.**(-magnitude/2.5))  # ph/s/cm^2/A
    #print('Input flux F_lambda = {:.3} erg/s/cm^2/A'.format(flux))
    
    # Collection area of SALT mirror
    area = lambda radius: np.pi*(radius**2)
    mirror = area(1000/2) - area(400/2)  # cm^2
    print('SALT mirror area = {:.3} cm^2'.format(mirror))
    
    # Incident photons per second on primary
    flux *= mirror # ph/s/A (theoretical)
    return flux

In [ ]:
target_flux = mag2flux(21.4)
print('Counts per second on primary = {:.5} erg/s/A'.format(target_flux))

SALT mirror area = 6.6e+05 cm^2
Counts per second on primary = 1.2756 erg/s/A


With the solid angle of the point spread function    
$\Omega = \pi \cdot FWHM^2 / 4$ arcsec$^2$

The diffuse input spectrum is converted from flux per square arcsecond to flux    
$F_\lambda = \Omega F_{\lambda\Omega}$

In [ ]:
sky_flux = mag2flux(20.9)
print('Counts per second on primary = {:.3} erg/s/A/arcsec^2'.format(sky_flux))

psf = np.pi * (fwhm / 2.)**2
print('Solid angle of the point spread function {:.3}'.format(psf))

# diffuse input
sky_flux = psf * (sky_flux / u.arcsec**2)  # ph/s/A
print('Sky contribution = {:.5} erg/s/A'.format(sky_flux))

SALT mirror area = 6.6e+05 cm^2
Counts per second on primary = 2.02 erg/s/A/arcsec^2
Solid angle of the point spread function 1.24 arcsec2
Sky contribution = 2.5029 erg/s/A


Scale of slit at telescope focal plane (in mm)    
$S_f = \frac{180 \cdot 3600}{\pi}\frac{1}{f_{tel}}\, [\mbox{''/mm}]$    
$f_{tel}$ the telescope focal length

In [ ]:
c = (180.*3600./np.pi)*u.arcsec
Sf = c/f_tel
print('Scale at the focal plane {:.3}'.format(Sf))

Scale at the focal plane 4.46 arcsec / mm


The collimator and camera reimage the incoming light, such that the image of the slit projected onto the CCD is reduced in size by the magnification factor
$M = \frac{f_{cam}}{f_{col}}$,
$f_{col}$ the collimator focal length and
$f_{cam}$ the camera focal length

Given a scale at the detector of
$S_d=S_f/M\, [\mbox{''/mm}]$    

In [ ]:
Sd = Sf * (f_col/f_cam)
print('Scale at the detector {:.3}'.format(Sd))

Scale at the detector 8.52 arcsec / mm


In [ ]:
phi_mm = phi_s/Sf  # mm
print('Linear width of the slit at the focal plane {:.3}'.format(phi_mm))

Linear width of the slit at the focal plane 0.336 mm


Waveband of a resolution element is determined by the the slit size projected onto the detector in angstroms:    
$\Delta\lambda_{re} = \frac{\phi_s}{S_f} \times \frac{\Lambda\cos(\alpha)}{f_{col}}\, [\mbox{A}]$    
$\phi_s$ the slit width in arcseconds, $\alpha$ the incident grading angle and $\Lambda$ the distance between grooves in angstroms with $1\,mm = 10^7\,A$

In [ ]:
# groove period (A)
L = (d/m).to(u.angstrom)  # A for waveband calculations
print('Grating period {:.3}'.format(L))

# step size is constant over wavelength
DLre = phi_mm*(np.cos(alpha)/f_col) * L  # A
print('Width of a resolution element {:.3}'.format(DLre))

# wave range
DL = (900.*u.nm - 360*u.nm).to(u.angstrom)
print('Total wave range {:.3}'.format(DL))

Grating period 1.11e+04 Angstrom
Width of a resolution element 5.79 Angstrom
Total wave range 5.4e+03 Angstrom


# Area of the spectral element

The photons arriving at the detector is dispersed over the geometrical extent of the resolution element in pixel space.

Width of the resolution element = $\frac{dx}{d\lambda} \Delta \lambda_{re}\, [\mbox{mm}]$    
where the linear dispersion $\frac{dx}{d\lambda} = f_{cam}\frac{d\beta}{d\lambda} = \frac{f_{cam}}{\Lambda \cos(\beta)}\, [\mbox{mm/A}]$    
with $\beta = \arcsin(\sin(\alpha) - \frac{\lambda}{\Lambda}) \, [\mbox{rad}]$ the diffraction angle.

In [ ]:
# l -- wavelenght [nm]
# a -- grading indicent angle [rad]
# p -- grading period [A]
beta = lambda l, a, p: np.arcsin(l/p.to(u.nm) - np.sin(a))

In [ ]:
# grating diffraction angle [rad]
diff_ang = beta(360. * u.nm, alpha, L)  # wavelength input [nm]
# angular dispersion [rad/A]
dbdl = 1. / (L * np.cos(diff_ang))
print('Angular dispersion {:.3}'.format(dbdl))
# linear dispersion [mm/A]
dxdl = f_cam * dbdl
print('Linear dispersion {:.3}'.format(dxdl))
# width of the resolution element
wre = dxdl*DLre
print('Area of resolution element {:.3}'.format(wre))

Angular dispersion 9.05e-05 1 / Angstrom
Linear dispersion 0.0299 mm / Angstrom
Area of resolution element 0.173 mm


The area of the resolution element is the width of the resolution element in the dispersion direction multiplied by the width of the PSF in the cross-dispersion direction    
$A_{re} = \frac{dx}{d\lambda} \Delta \lambda_{re} \times \frac{FWHM}{S_d}\, [\mbox{mm}^2]$ 

In [ ]:
Are = wre*fwhm/Sd
print('Projected area of resolution element {:.3}'.format(Are))

Projected area of resolution element 0.0255 mm2


# Area of a binned pixel

Area of a binned pixel $A_b = A_pB_xB_y$ [mm$^2$]    
$A_p$ the area of a CCD pixel, $B_x$ and $B_y$ the $x$ and $y$ binning factors

In [ ]:
Ap = (px.to(u.mm))*(py.to(u.mm))  # mm^2
print('Area of a CCD pixel {}'.format(Ap))

Ab = Ap * bx * by
print('Area of a binned pixel {}'.format(Ab))

Area of a CCD pixel 0.000225 mm2
Area of a binned pixel 0.0009 mm2


In [ ]:
# binning scale factor
c = Ab / Are
print('Binning scale factor {:.3}'.format(c))

Binning scale factor 0.0354


# SNR calculation

Integrate incoming spectra over resolution element at the focal plane to get incoming count rate, preserving some wavelength dependence at the focal plane

In [ ]:
N_lambda = target_flux / (u.angstrom) / (u.second)  # counts/s/A
Ns_lambda = sky_flux / (u.angstrom) / (u.second)  # counts/s/A

Integrated counts    
$N_{re} = \sum\limits_{\Delta\lambda_{re}} N_\lambda d\lambda$ [counts/s/re]

In [ ]:
N_target = N_lambda * DLre  # counts/s/re
print('Integrated spectra over resoluton element {:.3} [counts/s/re]'.format(N_target.value))
N_sky = Ns_lambda * DLre  # counts/s/re
print('Sky contribution over resolution element {:.3} [counts/s/re]'.format(N_sky.value))

Integrated spectra over resoluton element 7.38 [counts/s/re]
Sky contribution over resolution element 14.5 [counts/s/re]


Counts per bin    
$N'_{re} = N_{re}\frac{A_b}{A_{re}}$ [counts/s/bin]

In [ ]:
N_target *= c  # counts/s/bin
N_sky *= c  # counts/s/bin
print('Counts per bin signal {:.3} [counts/s/bin], sky {:.3} [counts/s/bin]'.format(N_target.value, N_sky.value))
N_total = N_target + N_sky
print('Total signal in counts per second per bin {:.3} [counts/s/bin]'.format(N_total.value))

Counts per bin signal 0.261 [counts/s/bin], sky 0.512 [counts/s/bin]
Total signal in counts per second per bin 0.773 [counts/s/bin]


## Noise contribution

Net exposure time    
$T_e = T - n_r T_r$ [s]    
the time spend counting photons are reduced by the time it takes to read out the CCD

In [ ]:
# total time on target
T = 1800 * u.second
# number of readout
nr = 1
# time per readout
ts = 4 * u.second
# readout noise in counts per readout bin
sr = 6

# net exposure time
Te = T - nr*ts
print('Net exposure time {}'.format(Te))

Net exposure time 1796.0 s


Noise per bin comes from three main sources: object, sky and readout

Noise for each signal component    
$\sigma = \sqrt{NT}\ $ [counts], with $N$ number of counts per bin and $T$ exposure time

In [ ]:
E_target = np.sqrt(N_target * Te)  # counts
E_sky = np.sqrt(N_sky * Te)  # counts
print('Noise contributions for object {:.3} and sky {:.3} [counts]'.format(E_target, E_sky))

Noise contributions for object 21.6 and sky 30.3 [counts]


Readout noise: $n_r\sigma_r^2$

In [ ]:
E_readout = nr * sr**2
print('Readout noise contribution {} [counts^2]'.format(E_readout))

Readout noise contribution 36 [counts^2]


Total noise per bin    
$\sigma^2 = \sigma^2_o + \sigma^2_s + n_r\sigma_r^2 = NT_e + n_r\sigma_r^2$

In [ ]:
signal = N_target * Te
noise = np.sqrt(N_total * Te + E_readout)
snr = signal/noise
print('Sensitivity = {:.3}/{:.3} = {:.3}'.format(signal, noise, snr))

Sensitivity = 4.69e+02/37.7 = 12.4
